In [1]:
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn as nn
import time
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import os
import natsort
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd
import numpy as np

# Function

In [2]:
class gpu_setting:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']
    
def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects

def loss_batch(loss_func, output, target, opt=None):
    loss = loss_func(output, target)
    metric_b = metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        output = model(xb)

        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)

        running_loss += loss_b
        
        if metric_b is not None:
            running_metric += metric_b
        
        if sanity_check is True:
            break

    loss = running_loss / len_data
    metric = running_metric / len_data

    return loss, metric

def train_val(model, params, epoch):
    num_epochs=epoch
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    val_dl=params["val_dl"]
    sanity_check=params["sanity_check"]
    lr_scheduler=params["lr_scheduler"]
    path2weights=params["path2weights"]

    loss_history = {'train': [], 'val': []}
    metric_history = {'train': [], 'val': []}

    # # GPU out of memoty error
    # best_model_wts = copy.deepcopy(model.state_dict())

    best_loss = float('inf')

    start_time = time.time()

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print('Epoch {}/{}, current lr={}'.format(epoch, num_epochs-1, current_lr))

        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl, sanity_check)
        loss_history['val'].append(val_loss)
        metric_history['val'].append(val_metric)

        if val_loss < best_loss:
            best_loss = val_loss
            # best_model_wts = copy.deepcopy(model.state_dict())

            torch.save(model.state_dict(), path2weights)
            print('Copied best model weights!')
            print('Get best val_loss')

        lr_scheduler.step(val_loss)

        print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
        print('-'*10)

    # model.load_state_dict(best_model_wts)

    return model, loss_history, metric_history

# Model

In [4]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        # identity mapping, input과 output의 feature map size, filter 수가 동일한 경우 사용.
        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        # projection mapping using 1x1conv
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=9, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [5]:
def ResNetParameters(model, train_dl, valid_dl):
    device = gpu_setting.device
    model = model.lower()
    if model == 'resnet34':
        model = resnet34().to(device)
    if model == 'resnet50':
        model = resnet50().to(device)
    if model == 'resnet101':
        model = resnet101().to(device)
        
    loss_func = nn.CrossEntropyLoss(reduction='sum')
    opt = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)
    
    # definc the training parameters
    params_train = {
        'num_epochs':21,
        'optimizer':opt,
        'loss_func':loss_func,
        'train_dl':train_dl, 
        'val_dl':valid_dl,
        'sanity_check':False,
        'lr_scheduler':lr_scheduler,
        'path2weights':'../Model_Weights/Reject_Option_res.pt',
    }
    return model, params_train

# Initial Data Load

In [6]:
class CustomSubset(Dataset):
    def __init__(self,Subset,transform=None):
        super(CustomSubset,self).__init__()
        self.Subset=Subset
        self.indices=Subset.indices
        self.transform=transform

    def __len__(self):
        return len(self.Subset)

    def __getitem__(self,idx):
        img,label=self.Subset[idx]
        if self.transform is not None:
            img=self.transform(img)
        return img,label

In [7]:
# Initial data
init_data_dir = '../Data/Labeled/'
init_data_folder_dataset = dset.ImageFolder(root=init_data_dir)

# train & validation
train_data_len = int(len(init_data_folder_dataset)*0.8)
valid_data_len = len(init_data_folder_dataset) - train_data_len
train_data, valid_data = random_split(init_data_folder_dataset, [train_data_len, valid_data_len])

In [9]:
print(len(train_data), len(valid_data))

13836 3459


In [10]:
init_data_transformation = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor()])

train_data = CustomSubset(train_data, init_data_transformation)
valid_data = CustomSubset(valid_data, init_data_transformation)

init_train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
init_valid_dl = DataLoader(valid_data, batch_size=32, shuffle=False)

# Initial Model Training

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
model, params_train = ResNetParameters('resnet50', init_train_dl, init_valid_dl)

In [15]:
model, loss_hist, metric_hist = train_val(model, params_train, 5)

Epoch 0/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.363113, val loss: 0.443182, accuracy: 90.08, time: 0.5816 min
----------
Epoch 1/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.244749, val loss: 0.201385, accuracy: 93.90, time: 1.1291 min
----------
Epoch 2/4, current lr=0.001
train loss: 0.204119, val loss: 0.230481, accuracy: 92.83, time: 1.6757 min
----------
Epoch 3/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.188822, val loss: 0.199593, accuracy: 94.07, time: 2.2236 min
----------
Epoch 4/4, current lr=0.001
Copied best model weights!
Get best val_loss
train loss: 0.168645, val loss: 0.146808, accuracy: 95.66, time: 2.7704 min
----------


# Test Data Load

In [36]:
class CustomDataSet():
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = all_imgs
        
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

In [45]:
test_data_dir = '../Data/Unlabeled/'
test_dataset = CustomDataSet(test_data_dir, transform=init_data_transformation)
test_dl = DataLoader(test_dataset, shuffle=False)

# Softmax

In [46]:
pred_list =[]
size = len(test_dl)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X in test_dl:
        X = X.to(device)
        pred = model(X)
        sft = torch.nn.functional.softmax(pred, dim=1)
        pred_list.append(sft)

In [47]:
pred_list

[tensor([[4.4316e-07, 1.2610e-09, 6.5670e-03, 5.4213e-04, 1.9673e-04, 4.5860e-04,
          4.7502e-09, 9.9224e-01, 2.3584e-13]], device='cuda:0'),
 tensor([[1.0290e-05, 7.0040e-08, 3.5295e-02, 3.8369e-02, 1.7740e-03, 5.0898e-03,
          2.3028e-06, 9.1946e-01, 4.4973e-10]], device='cuda:0'),
 tensor([[9.8652e-07, 2.1321e-09, 9.2127e-03, 8.1640e-03, 1.9095e-04, 8.8584e-04,
          7.8364e-08, 9.8155e-01, 4.8449e-12]], device='cuda:0'),
 tensor([[2.8080e-05, 7.9344e-05, 7.4497e-01, 2.2762e-01, 2.4352e-02, 1.5233e-03,
          1.1092e-03, 2.9136e-04, 2.2187e-05]], device='cuda:0'),
 tensor([[1.7033e-07, 3.2027e-11, 7.0847e-04, 4.5265e-04, 1.4490e-05, 4.4664e-05,
          3.5953e-09, 9.9878e-01, 1.1645e-14]], device='cuda:0'),
 tensor([[5.1379e-08, 2.4795e-11, 9.9063e-05, 5.9409e-06, 3.3529e-06, 2.2308e-05,
          2.6567e-10, 9.9987e-01, 1.1960e-15]], device='cuda:0'),
 tensor([[4.0236e-06, 1.0866e-09, 1.5489e-04, 3.5673e-06, 4.1165e-05, 8.1691e-05,
          4.2085e-09, 9.9971e-

In [48]:
len(pred_list)

155655

# Uncertainty 1 : Least Confidence

In [49]:
max_list = []
for i in range(0,155655):
    max_list.append(float(pred_list[i][0].max()))

print(max_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
cnt = 0
label_list = []
label = pd.read_csv('../../../csv/data_155655.csv', index_col=0)

all_imgs = os.listdir(test_data_dir)
for i in range(0,155655):
    if pred_list[i][0].max()<0.99:
        idx = int(all_imgs[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

In [54]:
real_label_list = list(label['failureNum'].values)

In [55]:
wrong_cnt = 0
for i in tqdm(range(0,155655)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|█████████████████████████████████████████████████████████████████████| 155655/155655 [00:00<00:00, 4220987.70it/s]


36532

In [62]:
engineer_label_count = []
engineer_label_count.append(cnt)

wrong_cnt_list = []
wrong_cnt_list.append(wrong_cnt) 

In [63]:
print(engineer_label_count, wrong_cnt_list)

[48574] [36532]


# Uncertainty 2 : Least Margin

In [65]:
cnt = 0
label_list = []
label = pd.read_csv('../../../csv/data_155655.csv', index_col=0)

all_imgs = os.listdir(test_data_dir)
for i in range(0,155655):
    if bool(pred_list[i][0].sort().values[-1] - pred_list[i][0].sort().values[-2] <= 0.98):
        idx = int(all_imgs[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

In [66]:
real_label_list = list(label['failureNum'].values)

In [67]:
wrong_cnt = 0
for i in tqdm(range(0,155655)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|█████████████████████████████████████████████████████████████████████| 155655/155655 [00:00<00:00, 3855392.11it/s]


36500

In [68]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

In [69]:
print(engineer_label_count, wrong_cnt_list)

[48574, 44921] [36532, 36500]


# Uncertainty 3 : Entropy

In [70]:
cnt = 0
label_list = []
label = pd.read_csv('../../../csv/data_155655.csv', index_col=0)

all_imgs = os.listdir(test_data_dir)
for i in range(0,155655):
    log_outputs = torch.log(pred_list[i][0])
    entropy = - torch.sum(pred_list[i][0] * log_outputs)
    if entropy> 0.05:
        idx = int(all_imgs[i][:-4])
        engineerlabel = int(label[label['index']==idx]['failureNum'])
        label_list.append(engineerlabel)
        cnt = cnt + 1
    else:
        label_list.append(int(pred_list[i][0].argmax()))

In [71]:
wrong_cnt = 0
for i in tqdm(range(0,155655)):
    if label_list[i] != real_label_list[i]:
        wrong_cnt = wrong_cnt + 1
        
wrong_cnt

100%|█████████████████████████████████████████████████████████████████████| 155655/155655 [00:00<00:00, 4462229.86it/s]


36575

In [72]:
engineer_label_count.append(cnt)
wrong_cnt_list.append(wrong_cnt)

# Result

In [73]:
print(engineer_label_count, wrong_cnt_list)

[48574, 44921, 55797] [36532, 36500, 36575]
